# Crevasse model with a single parameter set
This note book looks in detail at individual solutions.  Another notebook looks at ensembles of solutions over different parts of the parameter space.

In [1]:
# The following few lines of code are unethical and shouldn't be used every again. But they work.
import os
os.environ['PATH'] = "/home/bradlipovsky/anaconda3/envs/fenicsproject/bin:" + os.environ['PATH']
print(os.environ['PATH'])

/home/bradlipovsky/anaconda3/envs/fenicsproject/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin


In [2]:
from dolfin import *
from mshr import *
import matplotlib.pyplot as plt
# %matplotlib notebook
from time import perf_counter
import numpy as np
import pickle
import scipy.integrate as integrate
import scipy.special as special
from os import path

import crevasse as c # main module for this project
from importlib import reload
reload(c)

<module 'crevasse' from '/home/bradlipovsky/notebooks/Near-Front-Crevasses/crevasse.py'>

## Define baseline parameters

In [3]:
# Geometry: domain width, domain height,  crevasse location, crevasse width, crevasse height
geom = {'W':60000,'H':300,'Lc':1000, 'Wc':1, 'Hc': 5}
    
# Materials: Youngs modulus, poisson ratio, ice density, water density, gravity
mats = {'E':1e10, 'nu':0.3, 'rho':910, 'rhow':1024, 'g':9.81}

## Define the FEM solution

In [4]:
U_all,mesh = c.elasticity_solutions(case='full-minus-prestress',verbose=True,crevasse_location="surface",
                             geometry=geom,swell_amplitude=1.0, swell_phase=0.0,swell_forcing='everything')
U_bottom,mesh = c.elasticity_solutions(case='full-minus-prestress',verbose=True,crevasse_location="surface",
                             geometry=geom,swell_amplitude=1.0, swell_phase=0.0,swell_forcing='bottom only')
U_front,mesh = c.elasticity_solutions(case='full-minus-prestress',verbose=True,crevasse_location="surface",
                             geometry=geom,swell_amplitude=1.0, swell_phase=0.0,swell_forcing='front only')


UMFPACK V5.7.8 (Nov 9, 2018): ERROR: out of memory



RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to successfully call PETSc function 'KSPSolve'.
*** Reason:  PETSc error code is: 76 (Error in external library).
*** Where:   This error was encountered inside /home/conda/feedstock_root/build_artifacts/fenics-pkgs_1617882212586/work/dolfin/dolfin/la/PETScKrylovSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  7f46aeb0b296da5bbb1fb0845822a72ab9b09c55
*** -------------------------------------------------------------------------


In [ ]:
mats['mu'] = mats['E']/2./(1+mats['nu'])
mats['lmbda'] = mats['E']*mats['nu']/(1+mats['nu'])/(1-2*mats['nu'])

# U = U_front
U = U_all
# U = U_bottom

fig,ax=plt.subplots(figsize=(16,5))
# c=plot(sigma(U,mats['lmbda'],mats['mu'])[0,0],mode='color',vmin=00e3,vmax=300e3,cmap='viridis')
cc=plot(c.sigma(U,mats['lmbda'],mats['mu'])[0,1],mode='color',cmap='seismic',vmin=-50e3,vmax=50e3)
# cc=plot(U[0],mode='color',vmin=-10,vmax=10,cmap='seismic')

ax.set_xlim([0,3000])
# ax.set_ylim([200,300])
plt.xlabel('Distance from ice front (m)')
plt.ylabel('Distance from bottom of ice shelf (m)')
plt.title('Shear stress (kPa)')
plt.colorbar(cc)
plt.show()

In [ ]:
Hw = mats['rho']/mats['rhow']*geom['H']
fig,ax=plt.subplots(figsize=(16,5))
plot(mesh)
ax.set_xlim([0,5e3])
plt.xlabel('Distance from ice front (m)')
plt.ylabel('Distance from bottom of ice shelf (m)')
plt.show()

# Does the big variation in SIF come from the front BC or the bottom BC?
## Answer: the bottom BC

Previous result:
#### Crevasse locations: 
(10270.676691729323, 10273.182957393483)
#### SIFs:
(638801.7601736342, 602083.4188574409)
SIF Difference is -6.098547 pct over 2.506266 m



In [ ]:
stop()
test_locations = (10273.182957393483,10270.676691729323)
KI=[]

for loc in test_locations:
    geom['Lc']= loc

    this_KI,this_KII = c.sif(geom,mats,verbose=False,loc='surface',swell_amplitude=1.0,
        swell_phase=3.5,swell_forcing='everything')
    
    KI.append(this_KI)

KI_front = []    
for loc in test_locations:
    geom['Lc']= loc

    this_KI,this_KII = c.sif(geom,mats,verbose=False,loc='surface',swell_amplitude=1.0,
        swell_phase=3.5,swell_forcing='front only')
    
    KI_front.append(this_KI)
    
KI_bottom = []    
for loc in test_locations:
    geom['Lc']= loc

    this_KI,this_KII = c.sif(geom,mats,verbose=False,loc='surface',swell_amplitude=1.0,
        swell_phase=3.5,swell_forcing='bottom only')
    
    KI_bottom.append(this_KI)

print(np.diff(KI)[0])
print(np.diff(KI_front)[0])
print(np.diff(KI_bottom)[0])

# Next question: is the mesh resolution fine enough within one swell wavelength of the crevasse?

In [ ]:
from time import perf_counter
D, flexural_gravity_wavelength = c.fgl(mats,geom)

KI_res_test = []

resolutions = np.logspace(-2,2,10) * flexural_gravity_wavelength

for i,loc in enumerate(test_locations):
    geom['Lc']= loc
    these_KIs = []
    for res in resolutions:
        t0 = perf_counter()
        this_KI,this_KII = c.sif(geom,mats,verbose=False,loc='surface',swell_amplitude=1.0,
            swell_phase=3.5,swell_forcing='bottom only',crevasse_refinement=res)
        t1 = perf_counter()
        dt=t1-t0
        print('Resolution: %d'%res)
        print('    KI=%f'%this_KI)
        print('    dt=%f'%dt)
        these_KIs.append(this_KI)
    KI_res_test.append (these_KIs)
    print(' ')

In [ ]:

resolutions = np.array(resolutions)
plt.subplots()
plt.plot(resolutions/flexural_gravity_wavelength,
         np.diff(KI_res_test,axis=0).flatten(),'-ok')
plt.grid()
plt.xscale('log')
# plt.yscale('log')

# Conclusion: the previous simulations didn't have high enough resolution near the crevasse. High resolution needs to be maintained within one flexural gravity wavelength of the crevasse.